# Complete Simulation Chain for Visibility Calculation

## Read in Config

In [1]:
import vipy.simulation.utils as ut
import vipy.layouts.layouts as layouts
import astropy.constants as const
from astropy import units as un
import time as t
import numpy as np

In [2]:
# rc = ut.read_config('/net/nfshome/home/sfroese/vipy/config/default.toml')
rc = ut.read_config("../config/test.toml")
array_layout = layouts.get_array_layout('eht')

src_crd = rc['src_coord']
wave1 = const.c/((float(rc['channel'].split(':')[0])-float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter
wave2 = const.c/((float(rc['channel'].split(':')[0])+float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter

## Generate (l,m)-plane / FOV

In [3]:
import vipy.simulation.scan as scan

In [4]:
grid = scan.create_bgrid(rc['fov_size'], 256, src_crd)
lm = scan.lm(grid, src_crd)

## Calculate Start and Stop times for every measurement

In [5]:
time = ut.calc_time_steps(rc)
print(time)

['2016:095:00:00:00.000' '2016:095:00:00:10.000' '2016:095:00:00:20.000'
 '2016:095:00:00:30.000' '2016:095:00:00:40.000' '2016:095:00:00:50.000'
 '2016:095:00:20:00.000' '2016:095:00:20:10.000' '2016:095:00:20:20.000'
 '2016:095:00:20:30.000' '2016:095:00:20:40.000' '2016:095:00:20:50.000'
 '2016:095:00:40:00.000' '2016:095:00:40:10.000' '2016:095:00:40:20.000'
 '2016:095:00:40:30.000' '2016:095:00:40:40.000' '2016:095:00:40:50.000'
 '2016:095:01:00:00.000' '2016:095:01:00:10.000' '2016:095:01:00:20.000'
 '2016:095:01:00:30.000' '2016:095:01:00:40.000' '2016:095:01:00:50.000'
 '2016:095:01:20:00.000' '2016:095:01:20:10.000' '2016:095:01:20:20.000'
 '2016:095:01:20:30.000' '2016:095:01:20:40.000' '2016:095:01:20:50.000'
 '2016:095:01:40:00.000' '2016:095:01:40:10.000' '2016:095:01:40:20.000'
 '2016:095:01:40:30.000' '2016:095:01:40:40.000' '2016:095:01:40:50.000'
 '2016:095:02:00:00.000' '2016:095:02:00:10.000' '2016:095:02:00:20.000'
 '2016:095:02:00:30.000' '2016:095:02:00:40.000' '2

## Calculate Baselines for one measurement

In [6]:
baselines = scan.get_baselines(src_crd, time[0:3], array_layout)

In [7]:
st1,st2 = scan.get_valid_baselines(baselines,28)
st1.shape

(24,)

## Calculate Mueller/Jones matrices

In [8]:
import numpy as np
import torch

In [9]:
%%time
start = t.time()
torch.set_num_threads(8)
JJ_f1 = scan.getJones(lm, baselines, wave1, time[0:3], src_crd, array_layout)
JJ_f2 = scan.getJones(lm, baselines, wave2, time[0:3], src_crd, array_layout)

CPU times: user 3.04 s, sys: 888 ms, total: 3.93 s
Wall time: 685 ms


## Open Source Image and compute Stokes vector

In [10]:
from astropy.io import fits
import matplotlib.pyplot as plt
import torch

In [11]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))
# plt.imshow((I[...,1]+I[...,2]).real)
# plt.colorbar()

## Integration

In [12]:
%%time
delta_t = rc['corr_int_time']
delta_f = float(rc['channel'].split(':')[1])*10**6
delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
delta_m = np.abs(lm[0,255,0]-lm[0,0,0])
print(delta_l)

integral = scan.integrate(JJ_f1, JJ_f2, I, 2, delta_t, delta_f, delta_l, delta_m)
integral.shape
end = t.time()
print(end - start)

0.0001830982550547064
1.044583797454834
CPU times: user 1.05 s, sys: 396 ms, total: 1.45 s
Wall time: 189 ms


In [13]:
integral.shape

torch.Size([12, 4])

In [14]:
integral

tensor([[ 1.5361e-04+1.9068e-04j,  1.1414e-04+1.4162e-04j,
          7.3597e-05+9.1274e-05j,  5.4686e-05+6.7790e-05j],
        [-2.8102e-05-3.3827e-05j, -5.9045e-06-7.1092e-06j,
         -2.1364e-04-2.5706e-04j, -4.4891e-05-5.4023e-05j],
        [-2.5165e-04-3.9236e-05j, -1.8750e-04-2.9215e-05j,
         -1.9086e-03-2.9781e-04j, -1.4220e-03-2.2175e-04j],
        [-4.0103e-05-3.6748e-05j, -1.9255e-05-1.7631e-05j,
         -3.0474e-04-2.7937e-04j, -1.4632e-04-1.3404e-04j],
        [ 8.5381e-04+1.6212e-03j,  1.7994e-04+3.4074e-04j,
          6.3850e-04+1.2062e-03j,  1.3456e-04+2.5352e-04j],
        [-5.4484e-04-4.4131e-04j, -1.1451e-04-9.2606e-05j,
         -2.6166e-04-2.1069e-04j, -5.4996e-05-4.4211e-05j],
        [ 5.9552e-04-4.8154e-04j,  4.4461e-04-3.5925e-04j,
          2.8795e-04-2.3250e-04j,  2.1498e-04-1.7346e-04j],
        [-4.0878e-05+8.8357e-05j, -8.6068e-06+1.8604e-05j,
         -3.0934e-04+6.6865e-04j, -6.5133e-05+1.4079e-04j],
        [-2.7928e-04+5.8789e-07j, -2.0811e-04+4.

## All scan loop

In [15]:
from tqdm import tqdm

In [16]:
torch.set_num_threads(12)

In [17]:
from dataclasses import dataclass
# stokes, IFs?

@dataclass
class Visibilities:
    I: [complex]
    Q: [complex]
    U: [complex]
    V: [complex]
    num: [int]
    scan: [int]
    base_num: [int]
    u: [float]
    v: [float]
    w: [float]
    date: [float]
    _date: [float]

    def __getitem__(self, i):
        baseline = Vis(
            self.I[i],
            self.Q[i],
            self.U[i],
            self.V[i],
            self.num[i],
            self.scan[i],
            self.base_num[i],
            self.u[i],
            self.v[i],
            self.w[i],
            self.date[i],
            self._date[i],
        )
        return baseline

    def get_values(self):
        return np.array([self.I, self.Q, self.U, self.V])
    
    def add(self, visibilities):
        self.I = np.concatenate([self.I, visibilities.I])
        self.Q = np.concatenate([self.Q, visibilities.Q])
        self.U = np.concatenate([self.U, visibilities.U])
        self.V = np.concatenate([self.V, visibilities.V])
        self.num = np.concatenate([self.num, visibilities.num])
        self.scan = np.concatenate([self.scan, visibilities.scan])
        self.base_num = np.concatenate([self.base_num, visibilities.base_num])
        self.u = np.concatenate([self.u, visibilities.u])
        self.v = np.concatenate([self.v, visibilities.v])
        self.w = np.concatenate([self.w, visibilities.w])
        self.date = np.concatenate([self.date, visibilities.date])
        self._date = np.concatenate([self._date, visibilities._date])
        

@dataclass
class Vis:
    I: complex
    Q: complex
    U: complex
    V: complex
    num: int
    scan: int
    base_num: int
    u: float
    v: float
    w: float
    date: float
    _date: float

In [18]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))

In [19]:
from vipy.simulation.scan import get_valid_baselines
visibilities = Visibilities([], [], [], [], [], [], [], [], [], [], [], [])
vis_num = np.zeros(1)
#i in total number of scans
for i in tqdm(range(30)):
    t = time[i*3:(i+1)*3]
    baselines = scan.get_baselines(src_crd, t, array_layout)
    
    valid = baselines.valid.reshape(-1, 28)
    mask = np.array(valid[:-1]).astype(bool) & np.array(valid[1:]).astype(bool)
    u = baselines.u.reshape(-1, 28)
    v = baselines.v.reshape(-1, 28)
    w = baselines.w.reshape(-1, 28)
    u_valid = u[:-1][mask]
    v_valid = v[:-1][mask]
    w_valid = w[:-1][mask]
    date = np.repeat(t[:-1].mjd.reshape(-1, 1), 28, axis=1)[mask]
    _date = np.zeros(len(u_valid))
    print(_date.shape)
    
    
    JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
    if JJ_f1.shape[0] == 1:
        continue
    JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)

    delta_t = rc['corr_int_time']
    delta_f = float(rc['channel'].split(':')[1])*10**6
    delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
    delta_m = np.abs(lm[0,255,0]-lm[0,0,0])
    
    vis_num = np.arange(JJ_f1.shape[2]//2) + 1 + vis_num.max()
    
    int_values = scan.integrate(JJ_f1, JJ_f2, I, 2, delta_t, delta_f, delta_l, delta_m)
    
    vis = Visibilities(
        int_values[:, 0],
        int_values[:, 1],
        int_values[:, 2],
        int_values[:, 3],
        vis_num,
        np.repeat(i+1, len(vis_num)),
        np.array([baselines[i].baselineNum() for i in range(len(vis_num))]),
        u_valid,
        v_valid,
        w_valid,
        date,
        _date,      
    )
    
    visibilities.add(vis)

  0%|          | 0/30 [00:00<?, ?it/s]

(12,)


  3%|▎         | 1/30 [00:00<00:24,  1.19it/s]

(12,)


  7%|▋         | 2/30 [00:01<00:23,  1.20it/s]

...

 93%|█████████▎| 28/30 [00:43<00:03,  1.88s/it]

(25,)


 97%|█████████▋| 29/30 [00:45<00:01,  1.79s/it]

(20,)


100%|██████████| 30/30 [00:46<00:00,  1.55s/it]


In [27]:
from vipy.fits.writer import create_hdu_list
from vipy.simulation.utils import read_config

In [28]:
conf = read_config("../config/default.toml")
conf

{'src_coord': <SkyCoord (ICRS): (ra, dec) in deg
     (187.70593076, 12.39112324)>,
 'fov_size': 0.00018382,
 'corr_int_time': 10.0,
 'scan_start': '2016:95:00:00:00',
 'scan_duration': 300,
 'scans': 72,
 'channel': '227297:4096',
 'interval_length': 1200}

In [29]:
hdu_list = create_hdu_list(visibilities, conf)

/home/kschmidt_local/.local/anaconda3/envs/vipy/lib/python3.9/site-packages/astropy/io/fits/fitsrec.py:699: UserWarning: Field 2 has a repeat count of 0 in its format code, indicating an empty field.
  warnings.warn(


In [31]:
hdu_list.writeto("test0.fits", overwrite=True)